In [ ]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cursor = conn.cursor()
cursor.execute("CREATE TABLE notes(id integer PRIMARY KEY, body text, title text)")
print(cursor.description)
conn.close()

None


In [11]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE my_db;")
conn.autocommit = False
cur.execute("CREATE SCHEMA my_schema")
conn.commit()
print(cur.description)
conn.close()



DuplicateDatabase: database "my_db" already exists


In [3]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
cur.execute("CREATE SCHEMA my_schema")
conn.commit()
print(cur.description)
conn.close()

DuplicateSchema: schema "my_schema" already exists


In [4]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [8]:
col_headers 

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [9]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


In [10]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [11]:
descriptions = get_col_set("boston.csv", 2);
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


In [12]:
descriptions

{'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.',
 'A&B ON POLICE OFFICER',
 'ABDUCTION - INTICING',
 'AFFRAY',
 'AIRCRAFT INCIDENTS',
 'ANIMAL ABUSE',
 'ANIMAL CONTROL - DOG BITES - ETC.',
 'ANIMAL INCIDENTS',
 'ANNOYING AND ACCOSTIN',
 'ANNOYING AND ACCOSTING',
 'ARSON',
 'ASSAULT & BATTERY',
 'ASSAULT & BATTERY D/W - KNIFE',
 'ASSAULT & BATTERY D/W - OTHER',
 'ASSAULT & BATTERY D/W - OTHER ON POLICE OFFICER',
 'ASSAULT - AGGRAVATED',
 'ASSAULT - AGGRAVATED - BATTERY',
 'ASSAULT - SIMPLE',
 'ASSAULT D/W - KNIFE ON POLICE OFFICER',
 'ASSAULT D/W - OTHER',
 'ASSAULT SIMPLE - BATTERY',
 'AUTO THEFT',
 'AUTO THEFT - LEASED/RENTED VEHICLE',
 'AUTO THEFT - MOTORCYCLE / SCOOTER',
 'AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON',
 'AUTO THEFT - RECOVERED IN BY POLICE',
 'AUTO THEFT LEASE/RENT VEHICLE',
 'AUTO THEFT OTHER',
 'B&E NON-RESIDENCE DAY - FORCIBLE',
 'B&E NON-RESIDENCE DAY - NO FORCE',
 'B&E NON-RESIDENCE DAY - NO PROP TAKEN',
 'B&E NON-RESIDENCE NIGHT - FORCE',
 'B&E RESIDENCE DAY - N

In [13]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [14]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [15]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');""")
conn.commit()


DuplicateObject: type "weekday" already exists


In [16]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
cur.execute("""
            CREATE TABLE my_schema.boston_crimes (
                incident_num integer PRIMARY KEY,
                offense_code integer,
                decription varchar(58),
                date date,
                week_day weekday,
                lat decimal(10, 8),
                long decimal(10, 8)
                );
             """)
conn.commit()
conn.close()

DuplicateTable: relation "boston_crimes" already exists


In [29]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
with open("boston.csv", "r") as f:
    cur.copy_expert("COPY my_schema.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    conn.commit()
conn.close()

In [30]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
cur.execute("SELECT * FROM my_schema.boston_crimes;")
print(cur.fetchone())

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))


In [34]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="20010802")
cur = conn.cursor()
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE my_db FROM public;")